<style>
    h1 {
        font-weight: bold;
        font-family: 'Times New Roman', Times, serif;
    }

    p {
        font-family: 'Times New Roman', Times, serif;
    }
</style>

<h1>Faktorët e performancës së studentëve</h1>

<p>Qëllimi i preprocesimit të datasetit <b>Faktorët e performancës së studentëve</b> është të strukturojë dhe pastrojë të dhënat për të lehtësuar analizën e thellë të ndikimeve në performancën akademike të studentëve. Ky proces siguron që të dhënat të jenë të pastra, të organizuara, dhe të gatshme për të zbuluar lidhjet kyçe që influencojnë suksesin e studentëve. Përmes kësaj analize të dhënash të përpunuara, mund të identifikohen dhe zbatohen strategji të efektshme arsimore që ndihmojnë në përmirësimin e arritjeve të studentëve dhe në ngritjen e cilësisë së proceseve mësimore.</p>

<style>
    h3 {
        font-weight: bold;
        font-family: 'Times New Roman', Times, serif;
        margin-top: 10px;
        margin-bottom: 10px;
    }
</style>
<h3>Libraritë e përdorura</h3>

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from IPython.display import display
from sklearn.preprocessing import StandardScaler

<style>
    h3 {
        font-weight: bold;
        font-family: 'Times New Roman', Times, serif;
        margin-top: 10px;
        margin-bottom: 20px;
    }
</style>
<h3>Leximi i datasetit</h3>

In [2]:
# Dataframe kryesore
main_df = pd.read_csv("../dataset/StudentPerformanceFactors.csv")

# Dataframe i preprocesuar
pre_df = pd.read_csv("../dataset/StudentPerformanceFactors_new.csv")

<style>
    h2 {
        font-weight: bold;
        font-family: 'Times New Roman', Times, serif;
        margin-top: 10px;
        margin-bottom: 20px;
    }
</style>

<h2>FAZA E PARË</h2>

<style>
    h3 {
        font-weight: bold;
        font-family: 'Times New Roman', Times, serif;
        margin-top: 10px; 
        margin-bottom: 10px; 
    }
</style>
<h3>1. Kualiteti i të dhënave</h3>

<style>
    h4 {
        font-weight: bold;
        font-family: 'Times New Roman', Times, serif;
        margin-top: 10px;
        margin-bottom: 10px;
    }
</style>
<h4>1.1 Definimi i tipeve të të dhënave</h4>

In [8]:
print("Tipet e të dhënave: \n")
print(main_df.dtypes)

# Atributet kategorike dhe numerike
categorical_columns = main_df.select_dtypes(include=['object']).columns
numerical_columns = main_df.select_dtypes(include=['int64', 'float64']).columns

print("\nAtributet kategorike:", categorical_columns)
print("\nAtributet numerike :", numerical_columns)

Tipet e të dhënave: 

Hours_Studied                  int64
Attendance                     int64
Parental_Involvement          object
Access_to_Resources           object
Extracurricular_Activities    object
Sleep_Hours                    int64
Previous_Scores                int64
Motivation_Level              object
Internet_Access               object
Tutoring_Sessions              int64
Family_Income                 object
Teacher_Quality               object
School_Type                   object
Peer_Influence                object
Physical_Activity              int64
Learning_Disabilities         object
Parental_Education_Level      object
Distance_from_Home            object
Gender                        object
Exam_Score                     int64
dtype: object

Atributet kategorike: Index(['Parental_Involvement', 'Access_to_Resources',
       'Extracurricular_Activities', 'Motivation_Level', 'Internet_Access',
       'Family_Income', 'Teacher_Quality', 'School_Type', 'Peer_Influen

<style>
    h4 {
        font-weight: bold;
        font-family: 'Times New Roman', Times, serif;
        margin-top: 10px;
        margin-bottom: 10px;
    }
</style>
<h4>1.2 Menaxhimi vlerave të zbrazëta (null):</h4>


In [11]:
# Metoda për gjetjen e vlerave null
null_counts = main_df.isnull().sum()

print(null_counts)
has_nulls = main_df.isnull().any()

print('\nAtributet që kanë vlera të zbrazëta (null):', ', '.join(has_nulls[has_nulls].index))

Hours_Studied                  0
Attendance                     0
Parental_Involvement           0
Access_to_Resources            0
Extracurricular_Activities     0
Sleep_Hours                    0
Previous_Scores                0
Motivation_Level               0
Internet_Access                0
Tutoring_Sessions              0
Family_Income                  0
Teacher_Quality               78
School_Type                    0
Peer_Influence                 0
Physical_Activity              0
Learning_Disabilities          0
Parental_Education_Level      90
Distance_from_Home            67
Gender                         0
Exam_Score                     0
dtype: int64

Atributet që kanë vlera të zbrazëta (null): Teacher_Quality, Parental_Education_Level, Distance_from_Home


<style>
    p {
        font-family: 'Times New Roman', Times, serif;
        margin-top: 10px;
        margin-bottom: 10px;
    }
</style>
<p>Në kodin më poshtë, kemi përpunuar të dhënat duke zëvëndësuar vlerat null me vlera të përshtatshme në varësi të tipit të atributit (kolonës). Fillimisht kemi bërë identifikimin e kolonave me vlera të zbrazëta të cilat ndahen në dy grupe: numerike dhe kategorike. Kolonat numerike zëvëndësohen me medianën për të shmangur ndikimin e outliers, ndërsa kolona të tjera numerike përdorin mesataren. Për kolonat kategorike, vlerat null zëvëndesohen me vlerën më të shpeshtë. Pas kësaj kontrollohet dataset-i për të siguruar që të gjitha vlerat <b>null</b> janë zëvëndësuar duke lejuar modifikimin direkt të të dhënave me <b>inplace=True</b> dhe duke parandaluar kopjet e padëshiruara.</p>

In [12]:
# Ndarja e kolonave në numerike dhe kategorike
numeric_cols = pre_df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = pre_df.select_dtypes(include=['object', 'category']).columns

# Për kolonat numerike vendos medianen si vlerë në Exam_Score
pre_df.fillna({'Exam_Score': pre_df['Exam_Score'].median()}, inplace=True)

# Vendos vlerën mesatare në kolonat e tjera numerike
for col in numeric_cols:
    if col != 'Exam_Score':
        pre_df.fillna({col: pre_df[col].mean()}, inplace=True)

# Për kolonat kategorike plotëso vlerat e zbrazëta me vlerën më të shpeshtë
for col in categorical_cols:
    pre_df.fillna({col: pre_df[col].mode()[0]}, inplace=True)

# Kontrollo përsëri për vlerat e zbrazëta për të konfirmuar
print("Atributet që kanë vlera null:")
print(pre_df.isnull().sum())


Atributet që kanë vlera null:
Hours_Studied                 0
Attendance                    0
Parental_Involvement          0
Access_to_Resources           0
Extracurricular_Activities    0
Sleep_Hours                   0
Previous_Scores               0
Motivation_Level              0
Internet_Access               0
Tutoring_Sessions             0
Family_Income                 0
Teacher_Quality               0
School_Type                   0
Peer_Influence                0
Physical_Activity             0
Learning_Disabilities         0
Parental_Education_Level      0
Distance_from_Home            0
Gender                        0
Exam_Score                    0
dtype: int64


In [14]:
# Lista e kolonave që duhet kontrolluar për vlera të zbrazëta
kolonat = [
    'Hours_Studied', 
    'Attendance', 
    'Parental_Involvement', 
    'Access_to_Resources', 
    'Extracurricular_Activities', 
    'Sleep_Hours', 
    'Previous_Scores', 
    'Motivation_Level', 
    'Internet_Access', 
    'Tutoring_Sessions', 
    'Family_Income', 
    'Teacher_Quality', 
    'School_Type', 
    'Peer_Influence', 
    'Physical_Activity', 
    'Learning_Disabilities', 
    'Parental_Education_Level', 
    'Distance_from_Home', 
    'Gender', 
    'Exam_Score'
]

# Largo rreshtat me vlera null në kolonat e specifikuara më lartë
pre_df.dropna(subset=kolonat, inplace=True)

# Shfaqni numrin e vlerave null që kanë mbetur në çdo kolonë
print(pre_df.isnull().sum())


Hours_Studied                 0
Attendance                    0
Parental_Involvement          0
Access_to_Resources           0
Extracurricular_Activities    0
Sleep_Hours                   0
Previous_Scores               0
Motivation_Level              0
Internet_Access               0
Tutoring_Sessions             0
Family_Income                 0
Teacher_Quality               0
School_Type                   0
Peer_Influence                0
Physical_Activity             0
Learning_Disabilities         0
Parental_Education_Level      0
Distance_from_Home            0
Gender                        0
Exam_Score                    0
dtype: int64


Identifikimin e duplikimeve (duplicates)

In [60]:
# Funksioni per identifikimin e duplikimeve
print("Duplikimet e gjetura: " + str(pre_df.duplicated().sum()))

Duplikimet e gjetura: 0


Ne dataset-in tone nuk kemi gjetur ndonje duplikate prandaj nuk ishte e nevojshme te bejme asnje fshirje te duplikateve. Kjo konfirmon kualitetin e te dhënave dhe na lejon te vazhdojme me analizen e metejeshme.

### 2. Zgjedhja e nen-bashksise se vetive

Duke perdorur datasetin e ri te gjeneruar, fillojme me percaktimin e vetive me te rendsishme per analize, duke u fokusuar ne ato qe jane te lidhura ngushte me Exam_Score.

In [3]:
# Zgjedhja e nenbashkesise se vetive
features_selected = [
    'Hours_Studied', 
    'Attendance', 
    'Parental_Involvement', 
    'Access_to_Resources', 
    'Extracurricular_Activities', 
    'Sleep_Hours', 
    'Previous_Scores', 
    'Motivation_Level', 
    'Internet_Access', 
    'Tutoring_Sessions', 
    'Family_Income', 
    'Teacher_Quality', 
    'Peer_Influence', 
    'Physical_Activity', 
    'Gender'
]

# Krijojme nje dataframe te ri me vetite e zgjedhura
df_selected_features = pre_df[features_selected + ['Exam_Score']]

# Shfaqim disa rreshta nga dataframe-i i ri
print("Dataframe me vetite e zgjedhura:")
display(df_selected_features)

Dataframe me vetite e zgjedhura:


,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,Peer_Influence,Physical_Activity,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Positive,3,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Negative,4,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Neutral,4,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Negative,4,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Neutral,4,Female,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6602,25,69,High,Medium,No,7,76,Medium,Yes,1,High,Medium,Positive,2,Female,68
6603,23,76,High,Medium,No,8,81,Medium,Yes,3,Low,High,Positive,2,Female,69
6604,20,90,Medium,Low,Yes,6,65,Low,Yes,3,Low,Medium,Negative,2,Female,68
6605,10,86,High,High,Yes,6,91,High,Yes,2,Low,Medium,Positive,3,Female,68


Krijimi i vetive te reja

Nje prej vetive te cilat do te krijojme eshte Study_Value e cila eshte nje prodhim i Hours_Studied dhe Prevous_Scores. Ky atribut do te ndihmoj per te kuptuar se sa efektive jane order e studimit ne lidhje me rezultatet e meparshme.

In [18]:
# Krijimi i vetive te reja
df_selected_features.loc[:, 'Study_Value'] = df_selected_features['Hours_Studied'] * df_selected_features['Previous_Scores']

# Shfaqim disa rreshta nga dataframe-i per te kontrolluar Study_Value
print("Dataframe pas krijimit te Study_Value:")
display(df_selected_features[['Hours_Studied', 'Previous_Scores', 'Study_Value']].head())

Dataframe pas krijimit të Study_Value:


,Hours_Studied,Previous_Scores,Study_Value
0,23,73,1679
1,19,59,1121
2,24,91,2184
3,29,98,2842
4,19,65,1235


Nje tjeter veti e re do te jete Activity_Score e cila eshte nje shprehje per te kombinuar disa aktivitete fizike dhe jashteshkollore. Perdorim Physical_Activity dhe Extracurricular_Activity per te krijuar kete veti.

In [25]:
# Krijimi i Activity_Score bazuar ne aktivitetet fizike dhe jashteshkollore
df_selected_features.loc[:, 'Activity_Score'] = df_selected_features['Physical_Activity'] + df_selected_features['Extracurricular_Activities'].apply(lambda x: 1 if x == 'Yes' else 0)

# Shfaqim disa rreshta nga dataframe-i per te kontrolluar Activity_Score
print("Dataframe pas krijimit të Activity_Score:")
display(df_selected_features[['Physical_Activity', 'Extracurricular_Activities', 'Activity_Score']].head())

Dataframe pas krijimit të Activity_Score:


,Physical_Activity,Extracurricular_Activities,Activity_Score
0,3.0,No,3.0
1,4.0,No,4.0
2,4.0,Yes,5.0
3,4.0,Yes,5.0
4,4.0,Yes,5.0


Binarizimi i Sleep_Hours

Krijojme vetine e cila tregon nese nje student ka me shume se 7 ore gjume, duke e binarizuar ate. Kjo mund te ndihmoj per te kuptuar ndikimin e gjumit/pushimit ne performancen akademike.

In [27]:
# Krijimi i dataframe-it të ri me një kopje
df_selected_features = pre_df[features_selected + ['Exam_Score']].copy()

# Binarizimi i Sleep_Hours: 0 per me pak se 7 ore, 1 per 7 ore ose me shume
df_selected_features.loc[:, 'Sleep_Binary'] = df_selected_features['Sleep_Hours'].apply(lambda x: 1 if x >= 7 else 0)

# Shfaqim disa rreshta nga dataframe-i per te kontrolluar Sleep_Binary
print("Dataframe pas binarizimit të Sleep_Hours:")
display(df_selected_features[['Sleep_Hours', 'Sleep_Binary']].head())


Dataframe pas binarizimit të Sleep_Hours:


,Sleep_Hours,Sleep_Binary
0,7,1
1,8,1
2,7,1
3,8,1
4,6,0


Krijojme vetine Family_Income duke bere binarizimin per te krijuar nje veti qe tregon nese te ardhurat e familjes jane te larta os te ulta. Kjo mund te behet duke klasifikuar Family_Income neper disa kategori low, medium, high.

In [31]:
# Binarizimi i Family_Income: 0 per "Low", 1 per "Medium" dhe "High"
df_selected_features.loc[:, 'Income_Binary'] = df_selected_features['Family_Income'].apply(lambda x: 1 if x in ['Medium', 'High'] else 0)

# Shfaqim disa rreshta nga dataframe-i per te kontrolluar Income_Binary
print("Dataframe pas binarizimit të Family_Income:")
display(df_selected_features[['Family_Income', 'Income_Binary']].head())


Dataframe pas binarizimit të Family_Income:


,Family_Income,Income_Binary
0,Low,0
1,Medium,1
2,Medium,1
3,Medium,1
4,Medium,1


#### Kontrollimi i te dhenave te reja

In [33]:
# Shikoni strukturën e re të dataframe-it
print("Dataframe pas krijimit të vetive të reja:")
display(df_selected_features.head())

Dataframe pas krijimit të vetive të reja:


,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,Peer_Influence,Physical_Activity,Gender,Exam_Score,Sleep_Binary,Income_Binary
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Positive,3,Male,67,1,0
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Negative,4,Female,61,1,1
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Neutral,4,Male,74,1,1
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Negative,4,Male,71,1,1
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Neutral,4,Female,70,0,1


In [4]:
# Definimi i kolonave numerike
numerical_columns = pre_df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Inicializimi StandardScaler
scaler = StandardScaler()

# Normalizojme kolonat numerike
pre_df[numerical_columns] = scaler.fit_transform(pre_df[numerical_columns])

# Shfaqim disa rreshta nga dataframe pas normalizimit
print("Dataframe pas normalizimit të kolonave numerike:")
display(pre_df[numerical_columns].head())

Dataframe pas normalizimit të kolonave numerike:


,Hours_Studied,Attendance,Sleep_Hours,Previous_Scores,Tutoring_Sessions,Physical_Activity,Exam_Score
0,0.504942,0.348375,-0.019796,-0.143800,-1.213934,0.031411,-0.060578
1,-0.162822,-1.383736,0.661399,-1.116110,0.411451,1.001199,-1.602931
2,0.671882,1.560853,-0.019796,1.106313,0.411451,1.001199,1.738833
3,1.506587,0.781403,0.661399,1.592469,-0.401242,1.001199,0.967657
4,-0.162822,1.041220,-0.700990,-0.699406,1.224144,1.001199,0.710598


In [7]:
# Create a copy of the selected features DataFrame to avoid SettingWithCopyWarning
df_selected_features = df_selected_features.copy()

# Diskretizimi i Hours_Studied ne grupe
bins_hours_studied = [0, 10, 20, 30]
labels_hours_studied = ['Low', 'Medium', 'High']
df_selected_features['Hours_Studied_Binned'] = pd.cut(
    df_selected_features['Hours_Studied'],
    bins=bins_hours_studied,
    labels=labels_hours_studied,
    right=False
)

# Diskretizimi i Sleep_Hours ne grupe
bins_sleep_hours = [0, 6, 7, 8, 10]
labels_sleep_hours = ['Very Low', 'Low', 'Adequate', 'High']
df_selected_features['Sleep_Hours_Binned'] = pd.cut(
    df_selected_features['Sleep_Hours'],
    bins=bins_sleep_hours,
    labels=labels_sleep_hours,
    right=False
)

# Diskretizimi i Previous_Scores ne grupe
bins_previous_scores = [0, 60, 75, 90, 100]
labels_previous_scores = ['Low', 'Medium', 'High', 'Very High']
df_selected_features['Previous_Scores_Binned'] = pd.cut(
    df_selected_features['Previous_Scores'],
    bins=bins_previous_scores,
    labels=labels_previous_scores,
    right=False
)

# Shfaqim dataframe-in e azhurnuar me grupe te reja
print("Dataframe pas diskretizimit:")
display(df_selected_features[['Hours_Studied', 'Hours_Studied_Binned', 'Sleep_Hours', 'Sleep_Hours_Binned', 'Previous_Scores', 'Previous_Scores_Binned']].head())

Dataframe pas diskretizimit:


,Hours_Studied,Hours_Studied_Binned,Sleep_Hours,Sleep_Hours_Binned,Previous_Scores,Previous_Scores_Binned
0,23,High,7,Adequate,73,Medium
1,19,Medium,8,High,59,Low
2,24,High,7,Adequate,91,Very High
3,29,High,8,High,98,Very High
4,19,Medium,6,Low,65,Medium


In [6]:
# Krijojme variabla binare per kategorite e zgjedhura 
df_transformed = pd.get_dummies(df_selected_features, columns=['Gender', 'Parental_Involvement', 'Access_to_Resources', 'Motivation_Level', 'Internet_Access', 'Teacher_Quality', 'Peer_Influence', 'Family_Income', 'Hours_Studied_Binned', 'Sleep_Hours_Binned', 'Previous_Scores_Binned'])

# Shfaqim dataframe te transformuar
print("Dataframe pas transformimit:")
display(df_transformed.head())  

Dataframe pas transformimit:


,Hours_Studied,Attendance,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Tutoring_Sessions,Physical_Activity,Exam_Score,Gender_Female,Gender_Male,...,Hours_Studied_Binned_Medium,Hours_Studied_Binned_High,Sleep_Hours_Binned_Very Low,Sleep_Hours_Binned_Low,Sleep_Hours_Binned_Adequate,Sleep_Hours_Binned_High,Previous_Scores_Binned_Low,Previous_Scores_Binned_Medium,Previous_Scores_Binned_High,Previous_Scores_Binned_Very High
0,23,84,No,7,73,0,3,67,False,True,...,False,True,False,False,True,False,False,True,False,False
1,19,64,No,8,59,2,4,61,True,False,...,True,False,False,False,False,True,True,False,False,False
2,24,98,Yes,7,91,2,4,74,False,True,...,False,True,False,False,True,False,False,False,False,True
3,29,89,Yes,8,98,1,4,71,False,True,...,False,True,False,False,False,True,False,False,False,True
4,19,92,Yes,6,65,3,4,70,True,False,...,True,False,False,True,False,False,False,True,False,False
